# Fast-Trips Tutorial 3: User Classes

In [ ]:
import os,datetime
import pandas as pd

BASE_DIR         = r"PUT YOUR PATH TO THE TUTORIAL HERE"
BASE_DIR         = r"/Users/elizabeth/Documents/urbanlabs/MTC/SHRP2/tutorial"

One of the major benefits of using a disaggregate framework is that it allows extreme flexibility in market segmentation and accounting for heterogeneity.

One way that Fast-Trips accounts for heterogeneity is by using a trip-specific value-of-time in order to trade off costs with travel time.  This comes into play when fares are used.  Fares will be covered in a later tutorial.

The most straightforward way to account for market segmentation is to specifiy different `pathweights` for various subsets of trips.  This is done by:

  * categorizing trips by returning a user class in `config_ft.py`  
  * adding pathweight parameters for each named user class in `pathweights_ft.txt`
  * telling Fast-Trips which input function file to use by passing in `input_functions = config_ft.py` 
  * specifying which function in `config_ft.py` to use to determine the user class in `config_ft.txt` or passing it in as `user_class_function`
  


In this example, we will create a user class that is more sensitive to transfers than the others based on what household [school] they belong to.  Some people are just more....picky'n'choosy.

Let's review the new input files and how other input files have changed.

## `config_ft.py` specifies which trips belong in each user class

Here you can see that a function called `user_class` determines whether a trip is assigned to the picky user class based on their household.  

All other trips are assigned to an `other` user class.

In [ ]:
INPUT_FUNCTIONS  = os.path.join(BASE_DIR,"tta","input","demand-bigger","config_ft.py")

with open(INPUT_FUNCTIONS, 'r') as myfile:
    contents=myfile.read()
print contents

## `segmented_pathweight_ft.txt` specifies weights for multiple user classes

The same set of weights is specified for each user class. 

Notice different weights for `transfer_penalty` and the transfer `walk_time_min`


In [ ]:
INPUT_WEIGHTS    = os.path.join(BASE_DIR,"tta","input","demand-bigger","segmented_pathweight_ft.txt")
pathweights_df = pd.read_csv(INPUT_WEIGHTS)
pathweights_df[pathweights_df["supply_mode"]=="transfer"]

# Run Fast-Trips 
This example runs fast-trips 

In [ ]:
from fasttrips import Run

In [ ]:
USER_CLASS_FUNCT = "user_class"
RUN_CONFIG       = os.path.join(BASE_DIR,"tta","input","demand-bigger","config_ft.txt")

INPUT_NETWORKS   = os.path.join(BASE_DIR,"tta","input","network-added-route")
INPUT_DEMAND     = os.path.join(BASE_DIR,"tta","input","demand-bigger")

OUTPUT_FOLDER    = r"segmentation"
OUTPUT_DIR       = os.path.join(BASE_DIR, "tta","output")

In [ ]:
Run.run_fasttrips(input_network_dir= INPUT_NETWORKS,
                  input_demand_dir = INPUT_DEMAND,
                  run_config       = RUN_CONFIG,
                  input_weights    = INPUT_WEIGHTS,
                  output_dir       = OUTPUT_DIR,
                  output_folder    = OUTPUT_FOLDER,
                  user_class_function = USER_CLASS_FUNCT,
                  input_functions  = INPUT_FUNCTIONS,
                  iters            = 3,
                  overlap_variable = "count",
                  dispersion       = 0.5,
                  capacity         = True)

# Examine Results

## Vehicle/Route Level
First, let's examine what is going on with the vehicles. 

In [ ]:
pd.set_option('display.max_colwidth',160) 
full_output_directory=os.path.join(OUTPUT_DIR,OUTPUT_FOLDER)
vehicles_df = pd.read_csv(os.path.join(full_output_directory,"veh_trips.csv"), 
                                       sep=",", 
                                       parse_dates=['arrival_time', 'departure_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )

vehicles_df[['iteration','pathfinding_iteration','arrival_time','departure_time','route_id','trip_id','stop_id','boards','onboard','standees']]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set(style="whitegrid")
fig, axs = plt.subplots(nrows=3,figsize=(15,15))

iter1 = sns.barplot(ax=axs[0], x="trip_id", order=["B1","B2","B3","G1","R1","G2","R2","G3","G4"],y="boards", hue="pathfinding_iteration",  estimator=sum, ci=None, data=vehicles_df[vehicles_df["iteration"]==1])
iter2 = sns.barplot(ax=axs[1], x="trip_id", order=["B1","B2","B3","G1","R1","G2","R2","G3","G4"],y="boards", hue="pathfinding_iteration",  estimator=sum, ci=None, data=vehicles_df[vehicles_df["iteration"]==2])
iter3 = sns.barplot(ax=axs[2], x="trip_id", order=["B1","B2","B3","G1","R1","G2","R2","G3","G4"],y="boards", hue="pathfinding_iteration",  estimator=sum, ci=None, data=vehicles_df[vehicles_df["iteration"]==3])

axs[0].set_title('Iteration #1', fontsize=24,color="Gray")
axs[1].set_title('Iteration #2', fontsize=24,color="Gray")
axs[2].set_title('Iteration #3', fontsize=24,color="Gray")